# Excerise 4

In [227]:
import numpy as np
import pandas as pd
import os
import ast
import git
from git import RemoteProgress
from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [228]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

# Git Repo

In [229]:
#UAA
#remote_link = "https://github.com/cloudfoundry/uaa"
#local_link = "C:\\Users\\User\\repo\\uaa"
#repo = Repo(local_link)
#fixing_commit = "14350228989e2aee900b8d48a848293bb5152b6f"
#affected_files = ["server/src/main/resources/login-ui.xml"] 
#affected_files = ["uaa/src/test/java/org/cloudfoundry/identity/uaa/integration/feature/ChangePasswordIT.java"]
#affected_files = ["uaa/uaa/src/test/java/org/cloudfoundry/identity/uaa/mock/zones/DisableUserManagementSecurityFilterMockMvcTest.java"]


#Tomcat80
#remote_link = "https://github.com/apache/tomcat80"
#local_link = "C:\\Users\\User\\repo\\tomcat80"
#repo = Repo(local_link)
#fixing_commit = "c15c2aba8eb42425f9ebcfcaef579dada38ad3a2"
#affected_files = ["java/org/apache/catalina/Context.java"]
#affected_files = ["java/org/apache/catalina/core/StandardContext.java"]
#affected_files = ["java/org/apache/catalina/core/mbeans-descriptors.xml"]
#affected_files = ["java/org/apache/catalina/mapper/Mapper.java"]
#affected_files = ["java/org/apache/catalina/servlets/DefaultServlet.java"]
#affected_files = ["java/org/apache/catalina/startup/FailedContext.java"]
#affected_files = ["test/org/apache/catalina/core/TesterContext.java"]
#affected_files = ["test/org/apache/catalina/mapper/TestMapperWebapps.java"]
#affected_files = ["test/org/apache/catalina/startup/TomcatBaseTest.java"]
#affected_files = ["test/org/apache/catalina/valves/rewrite/TestRewriteValve.java"]
#affected_files = ["webapps/docs/changelog.xml"]
#affected_files = ["webapps/docs/config/context.xml"]


#Openjpa
#remote_link = "https://github.com/apache/openjpa"
#local_link = "C:\\Users\\User\\repo\\openjpa"
#repo = Repo(local_link)
#fixing_commit = "ad5cd6fb86af8809b367f709b6e041218055de2f"
#affected_files = ["openjpa-kernel/src/main/java/org/apache/openjpa/kernel/AbstractBrokerFactory.java"]
#affected_files = ["openjpa-lib/src/main/java/org/apache/openjpa/lib/conf/PluginValue.java"]
#affected_files = ["openjpa-lib/src/main/resources/org/apache/openjpa/lib/conf/localizer.properties"]
#affected_files = ["openjpa-slice/src/main/java/org/apache/openjpa/slice/jdbc/DistributedJDBCConfigurationImpl.java"]


# Function findStartEnd

In [230]:
def findStartEnd(file, line_affected):
    start = -1
    end = -1
    current_line = 1
    for line in file:
        if current_line == line_affected:
            i = current_line
            found = False
            while not found:
                i = i - 1
                if i == -1:
                    break
                if "{" in file[i - 1]:
                    found = True
                    start = i
                    break;
            found = False
            i = current_line
            while not found:
                i = i + 1
                if i >= (len(file) - 1):
                    break
                if "}" in file[i - 1]:
                    found = True
                    end = i
                    break   
        current_line = current_line + 1
    return start, end   

# Function makeList

In [231]:
def makeList(input_list, value):
    found = False
    for item in input_list:
        if item["commit"] == value:
            item["count"] = item["count"] + 1
            found = True
    if not found:
        input_list.append({"commit": value, "count": 1})
    return input_list

# Function findAddDelete

In [232]:
def findAddDelete(line):
    if line[0] == "@@":
        delete = line[1]
        delete = delete[1:]
        delete = delete.split(",")
        if len(delete) < 2:
            delete.append("1")
        
        add = line[2]
        add = add[1:]
        add = add.split(",")
        if len(add) < 2:
            add.append("1")
                
        return add, delete

# Function findCommon

In [233]:
def findCommon(commits_found, exclude):
    maximum = -1
    commit_found = None
    for commit in commits_found:
        if exclude in commit["commit"]:
            continue
        if commit["count"] > maximum:
            commit_found = commit["commit"]
            maximum = commit["count"]
            
    return commit_found, maximum

# Function getVCC

In [234]:
def getVCC(files, parameters):
    for file in files:
        diff_of_file = repo.git.diff(fixing_commit + "^" + ".." + fixing_commit, "-U0", "--", file).splitlines()
        diff_of_file = diff_of_file[4:]
        raw_commit_file = repo.git.show(fixing_commit + ":" + file).splitlines()
        commits_found = []
        i = 0
        while i < len(diff_of_file):
            split_line = diff_of_file[i].split()
            if split_line[0] == "@@":
                additions, deletions = findAddDelete(split_line)
                if deletions[1] != 0:
                    j = 0
                    line_number = int(deletions[0])
                    while j < int(deletions[1]):
                        j = j + 1
                        affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()
                        split_line = affected_file_diff[0].split()
                        makeList(commits_found, split_line[0])
                    i = i + j
                if additions[1] != 0:
                    j = 0
                    line_number = int(additions[0])
                    while j < int(additions[1]):
                        j = j + 1
                        start, end = findStartEnd(raw_commit_file, j + line_number)                         
                        if start != -1 and end != -1:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(start) + "," + str(end), fixing_commit, "--", file).splitlines()                         
                        else:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()             
                        for line in affected_file_diff:
                                split_line = line.split()
                                makeList(commits_found, split_line[0])
                        line_number = line_number + 1
                    i = i + j                
            i = i + 1
    return findCommon(commits_found, fixing_commit)

## Get VCC ID

In [235]:
parameters = None
# parameters = "-w"
# parameters = "-wM"
# parameters = "-wC"
# parameters = "-wCC"
# parameters = "-wCCC"

VCC = getVCC(affected_files, parameters)
print("VCC Commit:", VCC[0])

VCC Commit: af47a8b79b0fa07ff2c038197a07c09b745802ac


## Questions Answers

In [ ]:
show_data = repo.git.show("-s", vcc_commit).splitlines()
for line in show_data:
    print(line)
    
show_data = repo.git.show("-s", fixing_commit).splitlines()
for line in show_data:
    print(line)

In [ ]:
def convertToDate(input_date):
    split = input_date.split()
    year = split[4]
    month = strptime(split[1],'%b').tm_mon
    day = split[2]
    return date(year = int(year), month = month, day = int(day))

for affected_file in affected_files:
    dates = repo.git.log("--format=%ad", vcc_commit, "--", affected_file).splitlines()
    if len(dates) <= 0:
        print("File not in this commit: ", affected_file)
        continue
    date1_object = convertToDate(dates[0])
    if len(dates) < 2:
        print("No changes made")
    else:    
        date2_object = convertToDate(dates[1])
        total_days = abs((date1_object - date2_object).days)
        print("Total days since file", os.path.basename(affected_file), "changed:", total_days)